In [32]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np

path = "/Users/claireconzelmann/Documents/GitHub/KIHC-affordable-housing-analysis"
tif_districts_gdf = gpd.read_file(os.path.join(path, "Data/Processed/tif_districts.shp"))
metra_lines_gdf = gpd.read_file(os.path.join(path, "Data/Raw/MetraLinesshp.shp"))
l_lines = pd.read_csv(os.path.join(path, "Data/Raw/CTA_l_lines.csv"))
bus_routes_gdf = gpd.read_file(os.path.join(path, "Data/Processed/bus_routes.shp"))
etod_lots_tifs = gpd.read_file(os.path.join(path, "Data/Processed/etod_lots_tifs.shp"))
neighborhoods = pd.read_csv(os.path.join(path, "Data/Raw/Neighborhoods.csv"))
unit_area = pd.read_csv(os.path.join(path, "Data/Raw/zone min unit area.csv"))

In [33]:
#merge FAR and min unit area info
etod_lots_tifs.replace({"sq_ft": 0.0}, np.nan, inplace=True)
etod_lots_tifs = pd.merge(etod_lots_tifs, unit_area, on="zoning", how="outer")

In [34]:
#assume 20% of all lot square footage cannot be used for unit calculation
etod_lots_tifs["sq_ft_rentable"] = etod_lots_tifs["sq_ft"]*0.8

#update sqft based on far
etod_lots_tifs["sq_ft_far"] = etod_lots_tifs["sq_ft_rentable"]*etod_lots_tifs["FAR"]

#for non residential zoned lots, calculate sq footage above ground floor
etod_lots_tifs["sq_ft_residential"] = np.where((etod_lots_tifs["zone_cat"]=="B-Business") |
                                               (etod_lots_tifs["zone_cat"]=="C-Commercial"), 
                                               etod_lots_tifs["sq_ft_far"] - etod_lots_tifs["sq_ft_rentable"], 
                                               etod_lots_tifs["sq_ft_far"])

#assume 720 sq. ft. average unit size unless min unit size is larger
etod_lots_tifs["avg_unit_size"] = np.where(etod_lots_tifs["lot_area_per_unit"] > 720, 
                                           etod_lots_tifs["lot_area_per_unit"], 720)

In [35]:
# calculate estimate of number of units per lot
# 0 units if residential eligible sq ft is smaller than minimum unit size
etod_lots_tifs["n_units"] = np.where(etod_lots_tifs["avg_unit_size"] > etod_lots_tifs["sq_ft_residential"], 0, np.nan)

# divide residential eligible sq ft by average unit size for all others and round down
etod_lots_tifs["n_units"] = np.where(etod_lots_tifs["n_units"].isna(), 
                                     np.floor(etod_lots_tifs["sq_ft_residential"]/etod_lots_tifs["avg_unit_size"]), 
                                     etod_lots_tifs["n_units"])

# 1 unit for single family
etod_lots_tifs["n_units"] = np.where(etod_lots_tifs["zoning"].isin(["RS-1", "RS-2", "RS-3"]), 1, etod_lots_tifs["n_units"])

In [36]:
# calculate average number of units by zone to impute for lots missing sqft info
avg_units_zone = etod_lots_tifs.groupby("zoning")["n_units"].mean().reset_index(name="imputed_n_units")
etod_lots_tifs = pd.merge(etod_lots_tifs, avg_units_zone, on="zoning", how="outer")

# impute
etod_lots_tifs["n_units"] = np.where(etod_lots_tifs["n_units"].isna(), 
                                     np.floor(etod_lots_tifs["imputed_n_units"]),
                                     etod_lots_tifs["n_units"])

In [37]:
etod_lots_tifs["n_units"].sum(skipna=True)

np.float64(35287.0)

In [17]:
#calculate number of lots by zone and by neighborhood
lots_by_zone_neigh = etod_lots_tifs.groupby(["Community", "zoning"]).size().reset_index(name="n_lots")

#calculate number of lots by neighborhood
lots_by_neigh = etod_lots_tifs.groupby(["Community"]).size().reset_index(name="n_lots_neigh")

#calculate number of lots by zone category and neighborhood
lots_by_zone_cat = etod_lots_tifs.groupby(["Community", "zone_cat"]).size().reset_index(name="n_lots_cat")


In [18]:
#create broader category for zones
zone_cats = {"B-Business":"B", 
             "C-Commercial":"C",
             "D-Downtown": "D",
             "PD-Planned Development":"PD",
             "R-Residential":"R"}

def map_category(item):
    for key, value in zone_cats.items():
        if item.startswith(value):  # Check if item starts with dictionary value
            return key
    return "Unknown"  # Default value if no match is found

lots_by_zone_neigh["zone_cat"] = lots_by_zone_neigh["zoning"].apply(map_category)

In [20]:
#merge all counts together
n_lots_neigh_zone = pd.merge(lots_by_zone_neigh, lots_by_neigh, on="Community", how="outer")
n_lots_neigh_zone = pd.merge(n_lots_neigh_zone, lots_by_zone_cat, on=["Community", "zone_cat"], how="outer")
n_lots_neigh_zone = n_lots_neigh_zone.sort_values(by=["Community", "zoning"])

In [21]:
n_lots_neigh_zone.to_csv(os.path.join(path, "Data/Processed/vacant_lots_zone_counts.csv"))